# Pipeline Produto 1 — Base TSBio (RAW → Processado → Catálogo + Base Consolidada)

Este notebook substitui os scripts CLI e roda **todo o pipeline** em ambiente Jupyter.

## O que ele faz
1. **Processa** `data/Indicadores/**.csv` → `data/Indicadores_processado_por_tema/<Categoria>/*.csv`
2. Gera **Catálogo** → `outputs/catalogo_indicadores_tsbio.(csv|xlsx)`
3. Gera **Base Consolidada (long/tidy)** → `outputs/base_consolidada_tsbio.csv`

> **Padrão de nome de arquivo bruto**:  
> `FONTE - TEMA - RECORTE.csv` (RECORTE é opcional)  
> Ex.: `IBGE Censo Agro 2017 - Área média (ha) - Municípios.csv`


## 0) Configuração rápida

Ajuste apenas as variáveis abaixo (pastas de entrada/saída).  
Depois rode as células na ordem.


In [5]:
from pathlib import Path

# =============================================================================
# CONFIGURAÇÃO (layout atual do seu projeto)
# =============================================================================

DATA_DIR = Path(r"C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data")

# Brutos
ROOT_RAW = DATA_DIR / "Indicadores"

# Processados (1 arquivo por TEMA, com todos os municípios TSBio)
OUT_PROCESSADO = DATA_DIR / "Indicadores_processado_por_tema"
OUT_PROCESSADO_CSV = OUT_PROCESSADO / "csv"
OUT_PROCESSADO_XLSX = OUT_PROCESSADO / "xlsx"

# Saídas gerais do pipeline (catálogo + documentação + bases consolidadas)
OUT_DIR = OUT_PROCESSADO / "outputs"

# Dicionário oficial (coloque em fas_tsbio\notebook\)
DICT_PATH = Path(r"C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\notebook\dicionario_nomes_oficial_tsbio.csv")

# =============================================================================
# CONTROLES (o TdR pediu CSV + Excel)
# =============================================================================
EXPORT_PROCESSADO_CSV = True     # gera CSV por tema (canônico)
EXPORT_PROCESSADO_XLSX = True    # gera XLSX por tema (entrega)
REQUIRE_FULL_COVERAGE = False    # se True, marca "incompleto" quando faltar município
EXPORT_PROCESSADO_XLSX_ENGINE = "openpyxl"

# Arquivos de saída (catálogo e documentação)
OUT_CATALOGO_CSV = OUT_DIR / "catalogo_indicadores_tsbio.csv"
OUT_CATALOGO_XLSX = OUT_DIR / "catalogo_indicadores_tsbio.xlsx"
OUT_DOC_MD = OUT_DIR / "_documentacao.md"
OUT_DOC_XLSX = OUT_DIR / "_documentacao.xlsx"

# =============================================================================
# BASE CONSOLIDADA (FULL/DASHBOARD) — recomendado PARQUET/CSV.GZ
# =============================================================================
OUTPUT_FORMAT = "parquet"   # "parquet" | "csv_gz" | "csv"
ONLY_NUMERIC_ROWS = True
DROP_REPEATED_TEXT = True

GENERATE_FULL_BASE = True
GENERATE_DASHBOARD_BASE = True

OUT_BASE_FULL_PARQUET = OUT_DIR / "base_consolidada_tsbio_full.parquet"
OUT_BASE_FULL_CSV_GZ  = OUT_DIR / "base_consolidada_tsbio_full.csv.gz"
OUT_BASE_DASH_PARQUET = OUT_DIR / "base_consolidada_tsbio_dashboard.parquet"
OUT_BASE_DASH_CSV_GZ  = OUT_DIR / "base_consolidada_tsbio_dashboard.csv.gz"

# =============================================================================
# DASHBOARD — seleção de indicadores
# =============================================================================
# Opção A (recomendada): crie/edite o arquivo curado com coluna dashboard = sim/não
CATALOGO_CURADO_PATH = OUT_DIR / "catalogo_indicadores_tsbio_curado.csv"
DASHBOARD_FLAG_COLUMN = "dashboard"
DASHBOARD_INDICADOR_IDS = []   # Opção B: lista manual
DASHBOARD_FALLBACK_MAX = 80

# =============================================================================
# PREPARO
# =============================================================================
OUT_PROCESSADO_CSV.mkdir(parents=True, exist_ok=True)
OUT_PROCESSADO_XLSX.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

assert ROOT_RAW.exists(), f"ROOT_RAW não encontrado: {ROOT_RAW}"
assert DICT_PATH.exists(), f"DICT_PATH não encontrado: {DICT_PATH}"

ROOT_RAW, OUT_PROCESSADO_CSV, OUT_PROCESSADO_XLSX, OUT_DIR


(WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/Indicadores'),
 WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/Indicadores_processado_por_tema/csv'),
 WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/Indicadores_processado_por_tema/xlsx'),
 WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/Indicadores_processado_por_tema/outputs'))

In [6]:
import re
import unicodedata
from collections import defaultdict
from typing import Dict, List, Tuple

import pandas as pd
from tqdm import tqdm


In [7]:
# =============================================================================
# CONFIG TSBio (6 territórios)
# =============================================================================

TSBIO = [
    {"territorio_id": 1, "territorio_nome": "Altamira", "CD_MUN": ["1500602","1500859","1501725","1504455","1505486","1507805","1508159","1508357"]},
    {"territorio_id": 2, "territorio_nome": "Macapá", "CD_MUN": ["1600212","1600303","1600253","1600238","1600535","1600600","1600154","1600055"]},
    {"territorio_id": 3, "territorio_nome": "Portel", "CD_MUN": ["1503101","1504505","1505809","1501105"]},
    {"territorio_id": 4, "territorio_nome": "Juruá-Tefé", "CD_MUN": ["1301654","1301803","1301407","1301506","1301951","1301001","1304203","1302207","1304260","1300029"]},
    {"territorio_id": 5, "territorio_nome": "Rio Branco-Brasiléia", "CD_MUN": ["1200401","1200708","1200252","1200104","1200054","1200138","1200807","1200450","1200013","1200385","1200179"]},
    {"territorio_id": 6, "territorio_nome": "Salgado-Bragantino", "CD_MUN": ["1508209","1508035","1507961","1507474","1507466","1507409","1507102","1506906","1506609","1506203","1506112","1506104","1505601","1505007","1504406","1504307","1504109","1503200","1502905","1502608","1502202","1501709","1501600","1500909"]},
]

CATEGORIA_TO_DIMENSAO = {
    "População": "socioeconômica",
    "Educação": "socioeconômica",
    "Domicílios": "socioeconômica",
    "Trabalho e Renda": "socioeconômica",
    "Mercado de Trabalho": "socioeconômica",
    "Economia": "socioeconômica",
    "Religião": "socioeconômica",
    "Indígenas": "socioeconômica",
    "Quilombola": "socioeconômica",
    "Índices": "socioeconômica",
    "Desmatamento": "ambiental",
    "Uso e Cobertura do Solo": "ambiental",
    "Fundiário": "ambiental",
    "Agropecuária": "produtiva",
    "Entorno Domicílios": "infraestrutura",
    "Assistência Social": "políticas_públicas",
    "Vulnerabilidade": "vulnerabilidades",
    "Favelas e Comunidades Urbanas": "vulnerabilidades",
}

# Separadores
CSV_SEP_DEFAULT = ";"
SEPS_CANDIDATES = [";", ",", "\t"]
OUT_SEP = ";"
OUT_ENCODING = "utf-8-sig"


In [8]:
# =============================================================================
# Utilidades — normalização / dicionário / CSV
# =============================================================================

def safe_filename(name: str) -> str:
    name = re.sub(r'[\\/:*?"<>|]+', "_", (name or "").strip())
    name = re.sub(r"\s+", " ", name).strip()
    return name or "_"

def slugify(s: str) -> str:
    s = (s or "").strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def zfill_mun(x) -> str:
    x = re.sub(r"\D+", "", str(x))
    return x.zfill(7) if x else ""

def parse_parts_from_filename(filename: str) -> Tuple[str, str, str]:
    """Extrai (fonte, tema, recorte) via separador seguro ' - '."""
    name = filename[:-4] if filename.lower().endswith(".csv") else filename
    parts = [p.strip() for p in name.split(" - ") if p.strip()]
    fonte = parts[0] if len(parts) >= 1 else ""
    tema = parts[1] if len(parts) >= 2 else (parts[0] if parts else "")
    recorte = parts[2] if len(parts) >= 3 else ""
    return fonte, tema, recorte

def build_indicador_id(categoria: str, fonte: str, tema: str, recorte: str = "") -> str:
    base = f"{slugify(categoria)}__{slugify(fonte)}__{slugify(tema)}"
    if recorte and slugify(recorte) not in ("municipios", "municípios"):
        base = f"{base}__{slugify(recorte)}"
    return base

def detect_csv_sep(path: Path, encoding: str) -> str:
    try:
        with open(path, "r", encoding=encoding, errors="replace") as f:
            header = f.readline()
            if header.lower().startswith("sep="):
                header = f.readline()
    except Exception:
        return CSV_SEP_DEFAULT

    if not header:
        return CSV_SEP_DEFAULT

    counts = {s: header.count(s) for s in SEPS_CANDIDATES}
    best = max(counts, key=counts.get)
    return best if counts[best] > 0 else CSV_SEP_DEFAULT

def read_csv_local(path: Path) -> pd.DataFrame:
    last_err = None
    for enc in ("utf-8-sig", "latin1"):
        try:
            sep = detect_csv_sep(path, enc)
            return pd.read_csv(path, sep=sep, encoding=enc)
        except Exception as e:
            last_err = e
            continue
    raise last_err

def load_dictionary(path: Path) -> Dict[str, str]:
    """Carrega dicionário oficial: normalized_synonym -> canonical."""
    if not path.exists():
        return {}
    df = pd.read_csv(path, encoding="utf-8-sig")
    syn_map: Dict[str, str] = {}
    for _, r in df.iterrows():
        canonical = str(r["canonical"]).strip()
        syns = str(r.get("synonyms", "") or "").split("|")
        for s in syns + [str(r.get("label_pt", "") or "")]:
            ss = slugify(s)
            if ss:
                syn_map[ss] = canonical
    return syn_map

_UNIT_SUFFIX_MAP = {
    "%": "perc",
    "percent": "perc",
    "porcentagem": "perc",
    "ha": "ha",
    "hectare": "ha",
    "hectares": "ha",
    "km2": "km2",
    "km²": "km2",
    "m2": "m2",
    "m²": "m2",
    "r$": "rs",
    "rs": "rs",
    "reais": "rs",
    "pessoas": "pessoas",
    "pessoa": "pessoas",
}

def _unit_to_suffix(unit_raw: str) -> str:
    u = (unit_raw or "").strip().lower()
    if not u:
        return ""
    u = u.replace(" ", "")
    u = u.replace("R$", "r$").replace("²", "2")
    if "/" in u:
        left, right = u.split("/", 1)
        left_s = _UNIT_SUFFIX_MAP.get(left, slugify(left))
        right_s = _UNIT_SUFFIX_MAP.get(right, slugify(right))
        return f"{left_s}_por_{right_s}".strip("_")
    return _UNIT_SUFFIX_MAP.get(u, slugify(u))

_UNIT_PATTERN = re.compile(r"^(.*?)[\s]*[\(\[\{]([^\)\]\}]+)[\)\]\}]\s*$")

def normalize_column_name(col: str, syn_map: Dict[str, str]) -> str:
    """
    1) Se bater no dicionário oficial -> canonical
    2) Senão: snake_case + unidade no sufixo se houver (ex.: '(ha)' -> '_ha')
    """
    col = (col or "").strip()
    if not col:
        return col

    s = slugify(col)
    if s in syn_map:
        return syn_map[s]

    m = _UNIT_PATTERN.match(col)
    unit_suffix = ""
    base = col
    if m:
        base = m.group(1).strip()
        unit_suffix = _unit_to_suffix(m.group(2).strip())

    base_slug = slugify(base)
    if unit_suffix and not base_slug.endswith("_" + unit_suffix):
        base_slug = f"{base_slug}_{unit_suffix}"
    return base_slug or s


## 1) Processar brutos → processados

Gera um CSV por indicador, já com colunas padronizadas e metadados adicionados.


In [9]:
def processar_indicadores(
    root_raw: Path,
    out_csv_root: Path,
    out_xlsx_root: Path,
    dict_path: Path,
    export_csv: bool = True,
    export_xlsx: bool = True,
    require_full_coverage: bool = False,
) -> pd.DataFrame:
    """Processa brutos -> 1 arquivo por TEMA (com todos os municípios).

    Regras principais:
    - Fonte = antes do 1º ' - '
    - Tema  = texto ENTRE o 1º e o 2º ' - '  (a "parte do meio")
    - Recorte (se existir) vira METADADO (coluna), mas NÃO vira arquivo separado.

    Saída:
    - out_csv_root/<Categoria>/<Tema>.csv
    - out_xlsx_root/<Categoria>/<Tema>.xlsx
    """

    syn_map = load_dictionary(dict_path)

    # mapa TSBio
    expected_muns = set()
    mun_to_tsbio: Dict[str, Tuple[int, str]] = {}
    for t in TSBIO:
        tid = int(t["territorio_id"])
        tname = str(t["territorio_nome"])
        for m in t["CD_MUN"]:
            mm = zfill_mun(m)
            expected_muns.add(mm)
            mun_to_tsbio[mm] = (tid, tname)

    csv_files = sorted(root_raw.rglob("*.csv"))
    print(f"CSV brutos encontrados: {len(csv_files)} em {root_raw}")

    # bucket por categoria+fonte+tema (recorte não separa arquivo)
    buckets: Dict[Tuple[str, str, str], List[pd.DataFrame]] = defaultdict(list)

    # para evitar colisão de nomes quando houver mesmo tema com fontes diferentes
    fontes_por_cat_tema: Dict[Tuple[str, str], set] = defaultdict(set)

    errors = []
    missing_cod_mun = []

    for p in tqdm(csv_files, desc="Lendo CSVs brutos"):
        try:
            rel = p.relative_to(root_raw)
            categoria = rel.parts[0] if len(rel.parts) >= 2 else "(raiz)"
        except Exception:
            categoria = "(raiz)"

        fonte, tema, recorte = parse_parts_from_filename(p.name)
        fontes_por_cat_tema[(categoria, tema)].add(fonte)

        # indicador_id ignora recorte por definição (1 arquivo por tema)
        indicador_id = build_indicador_id(categoria, fonte, tema, recorte="")

        try:
            df = read_csv_local(p)
        except Exception as e:
            errors.append((str(p), str(e)))
            continue

        # padroniza colunas via dicionário oficial
        col_map = {c: normalize_column_name(c, syn_map) for c in df.columns}
        df = df.rename(columns=col_map)

        # fallback extra para casos raros
        if "cod_municipio" not in df.columns:
            # tenta localizar algo parecido
            possible = [c for c in df.columns if c.endswith("municipio") or "mun" in c]
            if possible:
                # não renomeia automaticamente para evitar erro silencioso
                pass
            missing_cod_mun.append(str(p))
            continue

        # padroniza código do município
        df["cod_municipio"] = df["cod_municipio"].apply(zfill_mun)
        df = df[df["cod_municipio"].astype(str).str.len() > 0].copy()

        # filtro TSBio
        df = df[df["cod_municipio"].isin(expected_muns)].copy()
        if df.empty:
            continue

        # metadados
        df["categoria"] = categoria
        df["fonte"] = fonte
        df["tema"] = tema
        df["recorte"] = recorte
        df["indicador_id"] = indicador_id
        df["arquivo_origem"] = p.name

        # mapeamento TSBio
        df["territorio_id"] = df["cod_municipio"].map(lambda x: mun_to_tsbio.get(x, (None, None))[0])
        df["territorio_nome"] = df["cod_municipio"].map(lambda x: mun_to_tsbio.get(x, (None, None))[1])
        df = df[df["territorio_id"].notna()].copy()

        buckets[(categoria, fonte, tema)].append(df)

    report = []

    for (categoria, fonte, tema), dfs in buckets.items():
        big = pd.concat(dfs, ignore_index=True)

        present = set(big["cod_municipio"].dropna().unique())
        missing = sorted(list(expected_muns - present))
        status = "ok" if (not require_full_coverage or not missing) else "incompleto"

        # pastas por categoria
        cat_dir_csv = out_csv_root / safe_filename(categoria)
        cat_dir_xlsx = out_xlsx_root / safe_filename(categoria)
        cat_dir_csv.mkdir(parents=True, exist_ok=True)
        cat_dir_xlsx.mkdir(parents=True, exist_ok=True)

        # nome do arquivo: 1 arquivo por tema (mas evita colisões se houver o mesmo tema com fontes diferentes)
        if len(fontes_por_cat_tema[(categoria, tema)]) > 1:
            out_base = safe_filename(f"{tema} - {fonte}")
        else:
            out_base = safe_filename(tema)

        out_csv_path = cat_dir_csv / f"{out_base}.csv"
        out_xlsx_path = cat_dir_xlsx / f"{out_base}.xlsx"

        # ordenação de colunas (metadados primeiro)
        first_cols = [
            "indicador_id",
            "territorio_id", "territorio_nome",
            "categoria", "fonte", "tema", "recorte",
            "cod_municipio", "municipio_nome", "sigla_uf",
            "ano", "mes", "classe", "observacao",
            "arquivo_origem",
        ]
        cols = [c for c in first_cols if c in big.columns] + [c for c in big.columns if c not in first_cols]
        big = big[cols]

        # export
        if export_csv:
            big.to_csv(out_csv_path, index=False, sep=OUT_SEP, encoding=OUT_ENCODING)

        excel_path_info = None
        if export_xlsx:
            try:
                big.to_excel(out_xlsx_path, index=False, engine="openpyxl", sheet_name="dados")
                excel_path_info = str(out_xlsx_path)
            except Exception as e:
                excel_path_info = f"erro: {e}"
                print(f"⚠️ Falha ao exportar Excel ({out_xlsx_path}): {e}")

        report.append({
            "categoria": categoria,
            "fonte": fonte,
            "tema": tema,
            "status": status,
            "arquivo_csv": str(out_csv_path) if export_csv else None,
            "arquivo_excel": excel_path_info,
            "linhas": len(big),
            "n_presentes": len(present),
            "n_esperados": len(expected_muns),
            "faltando_cod_municipio": ",".join(missing) if missing else "",
            "n_colunas": len(big.columns),
        })

    rep_df = pd.DataFrame(report).sort_values(["status", "categoria", "fonte", "tema"])
    rep_path = out_csv_root.parent / "_relatorio_validacao.csv" if out_csv_root.name == "csv" else out_csv_root / "_relatorio_validacao.csv"
    rep_df.to_csv(rep_path, index=False, encoding=OUT_ENCODING)

    if errors:
        err_path = out_csv_root.parent / "_erros_leitura.csv" if out_csv_root.name == "csv" else out_csv_root / "_erros_leitura.csv"
        pd.DataFrame(errors, columns=["arquivo", "erro"]).to_csv(err_path, index=False, encoding=OUT_ENCODING)

    if missing_cod_mun:
        miss_path = out_csv_root.parent / "_sem_coluna_cod_municipio.csv" if out_csv_root.name == "csv" else out_csv_root / "_sem_coluna_cod_municipio.csv"
        pd.DataFrame({"arquivo": missing_cod_mun}).to_csv(miss_path, index=False, encoding=OUT_ENCODING)

    print(f"✅ Processamento concluído. Relatório: {rep_path}")
    return rep_df


In [10]:
rep_df = processar_indicadores(
    root_raw=ROOT_RAW,
    out_csv_root=OUT_PROCESSADO_CSV,
    out_xlsx_root=OUT_PROCESSADO_XLSX,
    dict_path=DICT_PATH,
    export_csv=EXPORT_PROCESSADO_CSV,
    export_xlsx=EXPORT_PROCESSADO_XLSX,
    require_full_coverage=REQUIRE_FULL_COVERAGE,
)
rep_df.head(10)


CSV brutos encontrados: 9773 em C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\Indicadores


Lendo CSVs brutos: 100%|██████████| 9773/9773 [03:53<00:00, 41.84it/s] 


✅ Processamento concluído. Relatório: C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\Indicadores_processado_por_tema\_relatorio_validacao.csv


,categoria,fonte,tema,status,arquivo_csv,arquivo_excel,linhas,n_presentes,n_esperados,faltando_cod_municipio,n_colunas
0,Agropecuária,IBGE PAM 2024,Lavoura Permanente,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,125970,65,65,,19
1,Agropecuária,IBGE PAM 2024,Lavoura Temporaria,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,109395,65,65,,19
2,Agropecuária,IBGE PEVS 2024,Produção Extração Vegetal,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,10471,65,65,,15
3,Agropecuária,IBGE PEVS 2024,Produção Silvicultura,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,69,5,65,"1200013,1200054,1200104,1200138,1200179,120025...",17
4,Agropecuária,IBGE PPM 2024,Efetivo Rebanhos,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,15838,65,65,,13
5,Agropecuária,IBGE PPM 2024,Producão Aquicultura,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,1892,54,65,"1302207,1501105,1502608,1503101,1504505,150500...",14
6,Agropecuária,IBGE PPM 2024,Produção Origem Animal,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,7171,65,65,,15
7,Agropecuária,IBGE PPM 2024,Produção Pecuária,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,2630,65,65,,13
8,Agropecuária,PRONAF,Cobertura do PRONAF,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,65,65,65,,14
9,Assistência Social,Observatório Cadastro Único,Cadastro Único,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,65,65,65,,20


In [11]:
from typing import Tuple

def gerar_documentacao(
    rep_df: pd.DataFrame,
    out_md: Path,
    out_xlsx: Path,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Gera documentação (MD + XLSX) a partir do relatório do processamento."""

    rows = []
    col_rows = []

    for _, r in rep_df.iterrows():
        csv_path = Path(r["arquivo_csv"]) if pd.notna(r.get("arquivo_csv")) and r.get("arquivo_csv") else None
        xlsx_path = r.get("arquivo_excel") if pd.notna(r.get("arquivo_excel")) else None

        cols = []
        if csv_path and csv_path.exists():
            try:
                cols = list(pd.read_csv(csv_path, sep=OUT_SEP, encoding=OUT_ENCODING, nrows=0).columns)
            except Exception:
                cols = []
        rows.append({
            "categoria": r.get("categoria", ""),
            "fonte": r.get("fonte", ""),
            "tema": r.get("tema", ""),
            "status": r.get("status", ""),
            "arquivo_csv": str(csv_path) if csv_path else "",
            "arquivo_excel": xlsx_path if isinstance(xlsx_path, str) else "",
            "linhas": r.get("linhas", ""),
            "n_colunas": len(cols) if cols else r.get("n_colunas", ""),
            "colunas": "; ".join(cols),
        })

        for c in cols:
            col_rows.append({
                "categoria": r.get("categoria", ""),
                "fonte": r.get("fonte", ""),
                "tema": r.get("tema", ""),
                "coluna": c,
            })

    doc_df = pd.DataFrame(rows).sort_values(["categoria", "fonte", "tema"])
    cols_df = pd.DataFrame(col_rows).sort_values(["categoria", "fonte", "tema", "coluna"])

    # ---- MD ----
    lines = []
    for (categoria,), gcat in doc_df.groupby(["categoria"]):
        lines.append(f"# {categoria}")
        for _, rr in gcat.iterrows():
            lines.append(f"## {rr['tema']}")
            if rr.get("fonte"):
                lines.append(f"- **Fonte:** {rr['fonte']}")
            if rr.get("arquivo_csv"):
                lines.append(f"- **CSV:** {rr['arquivo_csv']}")
            if rr.get("arquivo_excel"):
                lines.append(f"- **XLSX:** {rr['arquivo_excel']}")
            if rr.get("linhas") != "":
                lines.append(f"- **Linhas:** {rr['linhas']}")
            lines.append("- **Colunas:**")
            cols = [c.strip() for c in str(rr.get("colunas","")).split(";") if c.strip()]
            for c in cols:
                lines.append(f"  - {c}")
            lines.append("")
        lines.append("")

    out_md.write_text("\n".join(lines).strip() + "\n", encoding="utf-8")

    # ---- XLSX ----
    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
        doc_df.to_excel(writer, index=False, sheet_name="arquivos")
        cols_df.to_excel(writer, index=False, sheet_name="colunas")

    print("✅ Documentação gerada:")
    print(" -", out_md)
    print(" -", out_xlsx)

    return doc_df, cols_df

doc_df, cols_df = gerar_documentacao(rep_df, OUT_DOC_MD, OUT_DOC_XLSX)
doc_df.head(10)


C:\Users\luiz.felipe\AppData\Local\Temp\ipykernel_26812\3287141967.py:48: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (categoria,), gcat in doc_df.groupby(["categoria"]):


ValueError: too many values to unpack (expected 1)

## 2) Gerar base consolidada (long/tidy)

Transforma os CSVs processados em uma base única no formato long:
- `variavel`, `valor`, `valor_num`, `unidade`


In [ ]:
UNIT_SUFFIX_TO_UNIT = {
    "perc": "%",
    "ha": "ha",
    "km2": "km²",
    "m2": "m²",
    "rs": "R$",
    "pessoas": "pessoas",
}

def infer_unidade_from_variavel(variavel: str) -> str:
    if not variavel:
        return ""
    v = str(variavel).strip().lower()
    suf = v.split("_")[-1]
    return UNIT_SUFFIX_TO_UNIT.get(suf, "")

def identificar_colunas_valor(df: pd.DataFrame) -> List[str]:
    excluir = {
        "indicador_id", "territorio_id", "territorio_nome", "categoria",
        "fonte", "tema", "recorte", "dimensao_tdr",
        "cod_municipio", "municipio_nome", "sigla_uf",
        "ano", "mes", "classe", "observacao", "arquivo_origem",
    }
    cols_valor = []
    for col in df.columns:
        if col in excluir:
            continue
        if pd.api.types.is_numeric_dtype(df[col]):
            cols_valor.append(col)
            continue
        if df[col].dtype == "object":
            s = df[col].astype(str)
            numeric = pd.to_numeric(
                s.str.replace(".", "", regex=False)
                 .str.replace(",", ".", regex=False)
                 .str.replace("%", "", regex=False)
                 .str.replace(" ", "", regex=False),
                errors="coerce",
            )
            if numeric.notna().mean() > 0.5:
                cols_valor.append(col)
    return cols_valor

def transformar_para_long(df: pd.DataFrame) -> pd.DataFrame:
    id_cols = [c for c in [
        "territorio_id","territorio_nome","cod_municipio","municipio_nome","sigla_uf",
        "ano","mes","indicador_id","tema","categoria","dimensao_tdr","fonte",
        "recorte","classe","observacao","arquivo_origem"
    ] if c in df.columns]

    value_cols = identificar_colunas_valor(df)
    if not value_cols:
        return pd.DataFrame()

    melted = pd.melt(df, id_vars=id_cols, value_vars=value_cols, var_name="variavel", value_name="valor")

    s = melted["valor"].astype(str)
    melted["valor_num"] = pd.to_numeric(
        s.str.replace(".", "", regex=False)
         .str.replace(",", ".", regex=False)
         .str.replace("%", "", regex=False)
         .str.replace(" ", "", regex=False),
        errors="coerce",
    )

    melted["unidade"] = melted["variavel"].map(infer_unidade_from_variavel)

    order = [
        "territorio_id","territorio_nome","cod_municipio","municipio_nome","sigla_uf",
        "ano","mes",
        "indicador_id","tema","categoria","dimensao_tdr","fonte","recorte",
        "variavel","valor","valor_num","unidade",
        "classe","observacao","arquivo_origem",
    ]
    cols = [c for c in order if c in melted.columns] + [c for c in melted.columns if c not in order]
    return melted[cols]

def _aplicar_controles_tamanho(base: pd.DataFrame) -> pd.DataFrame:
    # mantém apenas linhas numéricas (melhor para dashboard)
    if "ONLY_NUMERIC_ROWS" in globals() and ONLY_NUMERIC_ROWS and "valor_num" in base.columns:
        base = base[base["valor_num"].notna()].copy()

    # tabela fato enxuta (puxa metadados do catálogo depois via indicador_id)
    if "DROP_REPEATED_TEXT" in globals() and DROP_REPEATED_TEXT:
        drop_cols = [
            "valor",  # redundante se valor_num existe
            "tema", "categoria", "dimensao_tdr", "fonte", "recorte", "arquivo_origem",
            "territorio_nome", "municipio_nome", "sigla_uf",
            "observacao",
        ]
        base = base.drop(columns=drop_cols, errors="ignore")
    return base

def _salvar_base(base: pd.DataFrame, kind: str = "FULL"):
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    fmt = OUTPUT_FORMAT if "OUTPUT_FORMAT" in globals() else "parquet"

    if kind.upper() == "DASHBOARD":
        parquet_path = OUT_BASE_DASH_PARQUET
        csv_gz_path = OUT_BASE_DASH_CSV_GZ
        csv_path = OUT_DIR / "base_consolidada_tsbio_dashboard.csv"
    else:
        parquet_path = OUT_BASE_FULL_PARQUET
        csv_gz_path = OUT_BASE_FULL_CSV_GZ
        csv_path = OUT_DIR / "base_consolidada_tsbio_full.csv"

    if fmt == "parquet":
        try:
            base.to_parquet(parquet_path, index=False)
            print(f"✅ Base {kind} (PARQUET) salva em: {parquet_path} (linhas={len(base)})")
            return
        except Exception as e:
            print(f"⚠️ Não consegui salvar PARQUET para {kind} (provável falta de pyarrow/fastparquet).")
            print("   Salvando em CSV.GZ como fallback.")
            base.to_csv(csv_gz_path, index=False, sep=OUT_SEP, encoding=OUT_ENCODING, compression="gzip")
            print(f"✅ Base {kind} (CSV.GZ) salva em: {csv_gz_path} (linhas={len(base)})")
            return

    if fmt == "csv_gz":
        base.to_csv(csv_gz_path, index=False, sep=OUT_SEP, encoding=OUT_ENCODING, compression="gzip")
        print(f"✅ Base {kind} (CSV.GZ) salva em: {csv_gz_path} (linhas={len(base)})")
    else:
        base.to_csv(csv_path, index=False, sep=OUT_SEP, encoding=OUT_ENCODING)
        print(f"✅ Base {kind} (CSV) salva em: {csv_path} (linhas={len(base)})")

def gerar_base_consolidada(root_processado: Path, filter_indicador_ids: list | None = None) -> pd.DataFrame:
    files = sorted(root_processado.rglob("*.csv"))
    print(f"Processados encontrados: {len(files)} em {root_processado}")

    out_parts = []
    for p in tqdm(files, desc="Consolidando"):
        try:
            df = read_csv_local(p)
        except Exception:
            continue

        # filtro de indicadores (se solicitado)
        if filter_indicador_ids is not None and "indicador_id" in df.columns:
            # indicador_id costuma ser constante no arquivo
            iid = str(df["indicador_id"].dropna().iloc[0]) if df["indicador_id"].notna().any() else ""
            if iid and iid not in set(filter_indicador_ids):
                continue

        df_long = transformar_para_long(df)
        if not df_long.empty:
            out_parts.append(df_long)

    if not out_parts:
        raise RuntimeError("Nenhum dado consolidado (sem colunas de valor detectadas ou filtro muito restritivo).")

    base = pd.concat(out_parts, ignore_index=True)
    base = _aplicar_controles_tamanho(base)
    return base


In [ ]:
# ====== 2.1) Gera base FULL (arquivo/consulta) ======
if GENERATE_FULL_BASE:
    base_full = gerar_base_consolidada(OUT_PROCESSADO_CSV, filter_indicador_ids=None)
    _salvar_base(base_full, kind="FULL")
else:
    base_full = None
    print("ℹ️ GENERATE_FULL_BASE=False (pulando base FULL)")

# ====== 2.2) Gera base DASHBOARD (subconjunto) ======
if GENERATE_DASHBOARD_BASE:
    # dashboard_ids foi definido na célula 3.1 (seleção)
    base_dash = gerar_base_consolidada(OUT_PROCESSADO_CSV, filter_indicador_ids=dashboard_ids)
    _salvar_base(base_dash, kind="DASHBOARD")
else:
    base_dash = None
    print("ℹ️ GENERATE_DASHBOARD_BASE=False (pulando base DASHBOARD)")

# Mostra amostra da base do dashboard
(base_dash if base_dash is not None else base_full).head()

## 3) Gerar catálogo de indicadores (CSV + XLSX)

O catálogo é uma tabela “dimensão” que complementa a base consolidada e ajuda no Looker Studio.


In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

def identificar_colunas_valor_cols(cols: List[str]) -> List[str]:
    excluir = {
        "indicador_id","territorio_id","territorio_nome","categoria","fonte","tema","recorte","dimensao_tdr",
        "cod_municipio","municipio_nome","sigla_uf","ano","mes","classe","observacao","arquivo_origem",
    }
    return [c for c in cols if c not in excluir]

def inferir_unidade(variaveis: List[str]) -> str:
    units = set()
    for v in variaveis:
        suf = str(v).split("_")[-1].lower()
        u = UNIT_SUFFIX_TO_UNIT.get(suf, "")
        if u:
            units.add(u)
    if len(units) == 1:
        return list(units)[0]
    if len(units) > 1:
        return "multiplas"
    return ""

def inferir_periodo(path: Path) -> str:
    try:
        df = read_csv_local(path)
        if "ano" not in df.columns:
            return ""
        anos = pd.to_numeric(df["ano"], errors="coerce").dropna()
        if anos.empty:
            return ""
        return f"{int(anos.min())}-{int(anos.max())}"
    except Exception:
        return ""

def inferir_frequencia_por_fonte(fonte: str) -> str:
    f = (fonte or "").lower()
    if "censo" in f:
        return "decenal"
    if "mapbiomas" in f or "inpe" in f:
        return "anual"
    if "rais" in f or "caged" in f:
        return "mensal/anual"
    return "a definir"

def criar_excel(path_xlsx: Path, df: pd.DataFrame, sheet_name: str = "catalogo"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheet_name

    header_fill = PatternFill("solid", fgColor="1F4E79")
    header_font = Font(color="FFFFFF", bold=True)
    center = Alignment(horizontal="center", vertical="center", wrap_text=True)
    left = Alignment(horizontal="left", vertical="top", wrap_text=True)
    thin = Side(style="thin", color="D9D9D9")
    border = Border(left=thin, right=thin, top=thin, bottom=thin)

    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=1):
        ws.append(row)
        if r_idx == 1:
            for c_idx in range(1, len(row) + 1):
                cell = ws.cell(row=r_idx, column=c_idx)
                cell.fill = header_fill
                cell.font = header_font
                cell.alignment = center
                cell.border = border
        else:
            for c_idx in range(1, len(row) + 1):
                cell = ws.cell(row=r_idx, column=c_idx)
                cell.alignment = left
                cell.border = border

    for col in ws.columns:
        max_len = 0
        col_letter = col[0].column_letter
        for cell in col:
            try:
                max_len = max(max_len, len(str(cell.value)))
            except Exception:
                pass
        ws.column_dimensions[col_letter].width = min(max(12, max_len + 2), 55)

    wb.save(path_xlsx)

def gerar_catalogo(root_processado: Path, out_csv: Path, out_xlsx: Path) -> pd.DataFrame:
    files = sorted(root_processado.rglob("*.csv"))
    registros = []

    for p in tqdm(files, desc="Catalogando"):
        categoria = p.parent.name

        try:
            df_head = read_csv_local(p).head(5)
        except Exception:
            continue

        indicador_id = str(df_head["indicador_id"].dropna().iloc[0]) if "indicador_id" in df_head.columns and df_head["indicador_id"].notna().any() else ""
        tema = str(df_head["tema"].dropna().iloc[0]) if "tema" in df_head.columns and df_head["tema"].notna().any() else ""
        fonte = str(df_head["fonte"].dropna().iloc[0]) if "fonte" in df_head.columns and df_head["fonte"].notna().any() else ""
        recorte = str(df_head["recorte"].dropna().iloc[0]) if "recorte" in df_head.columns and df_head["recorte"].notna().any() else ""
        dimensao_tdr = str(df_head["dimensao_tdr"].dropna().iloc[0]) if "dimensao_tdr" in df_head.columns and df_head["dimensao_tdr"].notna().any() else CATEGORIA_TO_DIMENSAO.get(categoria, "outros")

        variaveis = identificar_colunas_valor_cols(list(df_head.columns))
        unidade = inferir_unidade(variaveis)
        periodo = inferir_periodo(p)
        freq = inferir_frequencia_por_fonte(fonte)

        registros.append({
            "indicador_id": indicador_id,
            "categoria": categoria,
            "tema": tema,
            "fonte": fonte,
            "recorte": recorte,
            "dimensao_tdr": dimensao_tdr,
            "unidade": unidade,
            "nivel_territorial": "municipio",
            "periodo": periodo,
            "frequencia_update": freq,
            "anexo_ii": "não",  # marcar depois (ou criar regras)
            "arquivo_csv": str(p),
            "variaveis": ", ".join(variaveis[:80]),
            "n_variaveis": len(variaveis),
        })

    cat = pd.DataFrame(registros).sort_values(["categoria", "fonte", "tema"])
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    cat.to_csv(out_csv, index=False, encoding=OUT_ENCODING)
    criar_excel(out_xlsx, cat)

    print(f"✅ Catálogo salvo em: {out_csv}")
    print(f"✅ Catálogo (XLSX) salvo em: {out_xlsx}")
    return cat


In [ ]:
catalogo = gerar_catalogo(OUT_PROCESSADO_CSV, OUT_CATALOGO_CSV, OUT_CATALOGO_XLSX)
catalogo.head()

## 3.1) Selecionar quais indicadores entram no dashboard

Você tem 2 jeitos:

### Opção A (recomendada)
1. Rode a geração do catálogo (célula anterior)  
2. Copie `outputs/catalogo_indicadores_tsbio.csv` para `outputs/catalogo_indicadores_tsbio_curado.csv`  
3. No arquivo curado, adicione/edite a coluna `dashboard` e marque `sim` nos indicadores que quer no Looker.

### Opção B
Preencha a lista `DASHBOARD_INDICADOR_IDS` na célula de configuração.


In [ ]:
def _truthy(x) -> bool:
    if x is None:
        return False
    s = str(x).strip().lower()
    return s in {"1", "true", "t", "yes", "y", "sim", "s"}

def selecionar_indicadores_para_dashboard(catalogo_df: pd.DataFrame) -> list:
    # 1) Se existir catálogo curado, usa ele
    df = catalogo_df.copy()
    if "CATALOGO_CURADO_PATH" in globals() and CATALOGO_CURADO_PATH.exists():
        try:
            df = pd.read_csv(CATALOGO_CURADO_PATH, encoding=OUT_ENCODING)
            print(f"✅ Usando catálogo curado: {CATALOGO_CURADO_PATH}")
        except Exception as e:
            print("⚠️ Não consegui ler o catálogo curado, vou usar o catálogo gerado.")
            print(e)

    # 2) Se tiver coluna de flag (dashboard), usa
    flag_col = DASHBOARD_FLAG_COLUMN if "DASHBOARD_FLAG_COLUMN" in globals() else "dashboard"
    if flag_col in df.columns:
        sel = df[df[flag_col].map(_truthy)]
        ids = sel["indicador_id"].dropna().astype(str).unique().tolist()
        if ids:
            print(f"✅ Indicadores marcados para dashboard via coluna '{flag_col}': {len(ids)}")
            return ids

    # 3) Se tiver lista manual preenchida, usa
    if "DASHBOARD_INDICADOR_IDS" in globals() and isinstance(DASHBOARD_INDICADOR_IDS, list) and len(DASHBOARD_INDICADOR_IDS) > 0:
        ids = [str(x).strip() for x in DASHBOARD_INDICADOR_IDS if str(x).strip()]
        print(f"✅ Indicadores para dashboard via lista manual: {len(ids)}")
        return ids

    # 4) Fallback: tenta pegar anexo_ii == sim (se existir) ou os primeiros N
    if "anexo_ii" in df.columns:
        sel = df[df["anexo_ii"].astype(str).str.lower().isin({"sim", "s", "1", "true"})]
        ids = sel["indicador_id"].dropna().astype(str).unique().tolist()
        if ids:
            print(f"ℹ️ Fallback: usando anexo_ii=sim ({len(ids)})")
            return ids

    nmax = DASHBOARD_FALLBACK_MAX if "DASHBOARD_FALLBACK_MAX" in globals() else 80
    ids = df["indicador_id"].dropna().astype(str).unique().tolist()[:nmax]
    print(f"⚠️ Nenhum indicador marcado. Fallback: primeiros {len(ids)} indicadores (max={nmax}).")
    return ids

dashboard_ids = selecionar_indicadores_para_dashboard(catalogo)
pd.Series(dashboard_ids, name="indicador_id").to_csv(OUT_DIR / "indicadores_dashboard_ids.csv", index=False, encoding=OUT_ENCODING)
print(f"✅ Lista salva em: {OUT_DIR / 'indicadores_dashboard_ids.csv'}")
len(dashboard_ids)

## 4) Checagens rápidas (sanidade)

Você pode usar estas checagens antes de publicar no Looker Studio.


In [ ]:
print("Base consolidada:")
print(" - linhas:", len(base))
print(" - indicadores:", base["indicador_id"].nunique() if "indicador_id" in base.columns else None)
print(" - municipios:", base["cod_municipio"].nunique() if "cod_municipio" in base.columns else None)
print(" - anos:", (int(base["ano"].min()), int(base["ano"].max())) if "ano" in base.columns and pd.api.types.is_numeric_dtype(base["ano"]) else None)

print("\nCatálogo:")
print(" - linhas:", len(catalogo))
print(" - indicadores:", catalogo["indicador_id"].nunique() if "indicador_id" in catalogo.columns else None)

# Top 10 indicadores com mais linhas
if "indicador_id" in base.columns:
    base.groupby("indicador_id").size().sort_values(ascending=False).head(10)

## 5) Publicar no Looker Studio (recomendação)

- Fonte 1: `outputs/base_consolidada_tsbio.csv`
- Fonte 2: `outputs/catalogo_indicadores_tsbio.xlsx` (ou CSV)
- Relacionar por: `indicador_id`

> Dica: Use filtros por `territorio_nome`, `ano`, `dimensao_tdr` e `fonte`.
